In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# =========================
# 0) CARGA Y LIMPIEZA
# =========================
df = pd.read_csv("df_limpio.csv")


df = df.loc[:, ~df.columns.str.contains(r"\.\d+$")]

# =========================
# 1) FUNCIONES
# =========================
def sturges_bins(n: int) -> int:
    if n <= 1:
        return 1
    return int(np.ceil(1 + np.log2(n)))

def to_numeric_percent(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip().str.replace("%", "").str.replace(",", "")
    s = pd.to_numeric(s, errors="coerce")
    vals = s.dropna()
    if len(vals) > 0 and (vals <= 1).all():
        s = s * 100
    return s

def price_to_numeric(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s.astype(str).str.replace(r"[^0-9.]", "", regex=True), errors="coerce")

def bathrooms_text_to_float(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.replace("half-bath", "0.5", case=False)
    num = s.str.extract(r"(\d+(\.\d+)?)")[0]
    return pd.to_numeric(num, errors="coerce")

def bar_from_counts(counts: pd.Series, title: str, xlabel: str, outfile: str):
    plt.figure()
    counts.plot(kind="bar")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Conteo")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(outfile)
    plt.close()

def safe_name(name: str) -> str:
    return str(name).replace(" ", "_").replace("/", "_")

# =========================
# 2) PASO 3: CATEGÓRICAS
# =========================
# Tomamos 10 categóricas distintas
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

if "property_type" in cat_cols:
    cat_cols.remove("property_type")
if "room_type" in df.columns and "room_type" not in cat_cols:
    cat_cols.insert(0, "room_type")

cat_cols_10 = cat_cols[:10]

cat_summary_rows = []
for col in cat_cols_10:
    s = df[col]
    vc = s.value_counts(dropna=True)
    top_val = vc.index[0] if len(vc) else None
    top_freq = int(vc.iloc[0]) if len(vc) else 0
    miss_pct = round(100 * (1 - s.notna().sum() / len(s)), 2) if len(s) else np.nan

    cat_summary_rows.append({
        "variable": col,
        "n": int(s.notna().sum()),
        "n_unique": int(s.nunique(dropna=True)),
        "top": top_val,
        "top_freq": top_freq,
        "missing_%": miss_pct
    })

    # Guardar tabla y gráfica
    fname = safe_name(col)
    counts20 = vc.head(20)
    counts20.to_csv(f"freq_{fname}.csv", header=["count"])
    bar_from_counts(counts20, f"Frecuencias (top 20) - {col}", col, f"freq_{fname}.png")

cat_summary_df = pd.DataFrame(cat_summary_rows)
cat_summary_df.to_csv("resumen_categoricas.csv", index=False)

# =========================
# 3) STURGES
# =========================
targets = [
    "host_response_rate", "host_acceptance_rate", "host_total_listings_count",
    "accommodates", "bathrooms_text", "beds", "price", "maximum_nights_avg_ntm",
    "availability_365", "number_of_reviews", "review_scores_value", "reviews_per_month"
]

num_summary = []
for var in targets:
    if var not in df.columns:
        num_summary.append({"variable": var, "status": "NO_ENCONTRADA"})
        continue

    s = df[var]
    if var in ["host_response_rate", "host_acceptance_rate"]:
        s = to_numeric_percent(s)
    elif var == "price":
        s = price_to_numeric(s)
    elif var == "bathrooms_text":
        s = bathrooms_text_to_float(s)
    else:
        s = pd.to_numeric(s, errors="coerce")

    s = s.dropna()
    if len(s) < 2:
        num_summary.append({"variable": var, "status": "SIN_DATOS"})
        continue

    k = sturges_bins(len(s))
    cats = pd.cut(s, bins=k, include_lowest=True)
    freq = cats.value_counts(sort=False)

    fname = safe_name(var)
    freq.rename("count").to_csv(f"sturges_{fname}.csv")
    bar_from_counts(freq, f"Sturges - {var} (k={k})", "Intervalos", f"sturges_{fname}.png")

    num_summary.append({
        "variable": var, "status": "OK", "n": len(s), "bins": k,
        "min": s.min(), "max": s.max(),
        "mean": s.mean(), "median": s.median(), "std": s.std()
    })

num_summary_df = pd.DataFrame(num_summary)
num_summary_df.to_csv("resumen_sturges_numericas.csv", index=False)

# =========================
# 4) PRINTS DE VERIFICACIÓN
# =========================
print("\n===== PASO 3: CATEGÓRICAS =====")
print(cat_summary_df.to_string(index=False))

print("\n===== PASO 4: STURGES =====")
print(num_summary_df.to_string(index=False))

print("\n✅ Archivos CSV y PNG guardados en la carpeta actual.")


/tmp/ipython-input-1857152973.py:44: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()



===== PASO 3: CATEGÓRICAS =====
              variable   n  n_unique                                       top  top_freq  missing_%
    host_response_rate 568        22                                      100%       511      10.69
  host_acceptance_rate 601        54                                      100%       260       5.50
                 price 496       178                                    $90.00        11      22.01
         host_location 612        25                            Washington, DC       567       3.77
    host_response_time 568         4                            within an hour       406      10.69
     host_is_superhost 630         2                                         t       371       0.94
host_identity_verified 635         2                                         t       526       0.16
neighbourhood_cleansed 635        38 Union Station, Stanton Park, Kingman Park        80       0.16
             room_type 635         2                           Enti